In [3]:
import os
import json
import os.path as osp


def load_json(json_path):
    with open(json_path, 'r') as f:
        json_data = json.load(f)
    return json_data


data_root = '/data5/chence/FFHQ/FFHQ'
json_files = sorted([i for i in os.listdir(data_root) if i.endswith('.json') and i.startswith('meta_')])
print(json_files)

['meta_1-7.json', 'meta_2-7.json', 'meta_3-7.json', 'meta_4-7.json', 'meta_5-7.json', 'meta_6-7.json', 'meta_7-7.json']


In [4]:
merged_data = {}
for idx in range(len(json_files)):
    json_file = json_files[idx]
    with open(os.path.join(data_root, json_file), 'r') as f:
        data = json.load(f)
        print(len(data))
        merged_data.update(data)
print(len(merged_data))

10000
10000
10000
10000
10000
10000
9938
69938


In [3]:
meta_json_file = 'meta.json'
with open(os.path.join(data_root, meta_json_file), 'w') as f:
    json.dump(merged_data, f, indent=4)

In [5]:
landmark_json_file = '/data5/chence/FFHQ/FFHQ/ffhq-dataset-v2.json'
landmark_json_data = load_json(os.path.join(data_root, landmark_json_file))
print(len(landmark_json_data))

70000


In [6]:
from tqdm import tqdm

origin_lm = {}
for image, image_meta in tqdm(landmark_json_data.items()):
    img_path = image_meta["in_the_wild"]["file_path"].split('/')
    del img_path[1]
    img_path = '/'.join(img_path)
    origin_lm[img_path] = image_meta["in_the_wild"]["face_landmarks"]

100%|██████████| 70000/70000 [00:00<00:00, 411802.64it/s]


In [7]:
import torch
import sys
sys.path.insert(0, '..')
import ChamferDistancePytorch.chamfer2D.dist_chamfer_2D as dist_chamfer_2D
import numpy as np

chamLoss = dist_chamfer_2D.chamfer_2DDist()

Loaded compiled 2D CUDA chamfer distance


In [8]:
import copy

filtered_meta = {}

for merged_key in tqdm(list(merged_data.keys())):
    saved_lm_idx = []
    lmval = origin_lm[merged_key]
    lmk_names = []
    dists = []
    for lmk_ in merged_data[merged_key]['raw']['landmarks'].keys():
        if merged_data[merged_key]['head'][lmk_]['view'] == 'back': continue
        lmval_ = merged_data[merged_key]['raw']['landmarks'][lmk_]
        if lmval_ is None or lmval is None:
            continue
        dist1, dist2, idx1, idx2 = chamLoss(
            torch.from_numpy(np.array(lmval_, dtype=np.float32)).unsqueeze(0).cuda(),
            torch.from_numpy(np.array(lmval, dtype=np.float32)).unsqueeze(0).cuda()
        )
        lmk_names.append(lmk_)
        dists.append((np.mean(dist1.cpu().detach().numpy())+np.mean(dist2.cpu().detach().numpy()))/2.)
    if len(dists) != 0:
        sid = lmk_names[np.argmin(dists)]
        if merged_data[merged_key]['head'][sid]['view'] == 'front':
            dists = np.array(dists)
            saved_lm_idx.append(sid)
    saved_lm_idx = sorted(set(saved_lm_idx))
    if len(saved_lm_idx) != 0:
        filtered_meta[merged_key] = copy.deepcopy(merged_data[merged_key])
        filtered_meta[merged_key]['raw']['landmarks'] = {}
        filtered_meta[merged_key]['head'] = {}
        for slmidx, imkey in enumerate(saved_lm_idx):
            filtered_meta[merged_key]['raw']['landmarks'][f'{slmidx:02d}'] = merged_data[merged_key]['raw']['landmarks'][imkey]
            filtered_meta[merged_key]['head'][f'{slmidx:02d}'] = merged_data[merged_key]['head'][imkey]

100%|██████████| 69938/69938 [09:46<00:00, 119.28it/s] 


In [9]:
len(filtered_meta)

68766

In [10]:
import tqdm

for k in tqdm.tqdm(filtered_meta.keys()):
    for hk in filtered_meta[k]['head'].keys():
        if filtered_meta[k]['head'][hk]['view'] != 'front':
            print(k, hk)

100%|██████████| 68766/68766 [00:00<00:00, 124591.74it/s]


In [11]:
meta_hq_json_file = 'meta_hq.json'
with open(os.path.join(data_root, meta_hq_json_file), 'w') as f:
    json.dump(filtered_meta, f, indent=4)